In [4]:
# importing libraries
from __future__ import print_function
import sys
import os
import collections
import SimpleITK as sitk
from SimpleITK.SimpleITK import JoinSeries
import logging
import six
import radiomics
from radiomics import featureextractor, getFeatureClasses, getTestCase, firstorder, glcm, imageoperations, shape, glrlm, glszm
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import PIL
from PIL import Image
from google.colab import files
import pandas as pd
import csv

In [1]:
# install PyRadiomics if not installed already
!pip install pyradiomics

     |████████████████████████████████| 188 kB 5.2 MB/s 
     |████████████████████████████████| 48.4 MB 20 kB/s 
     |████████████████████████████████| 109 kB 52.3 MB/s 
     |████████████████████████████████| 546 kB 42.2 MB/s 


In [5]:
!wget --no-check-certificate \
    "https://github.com/lukevrobbins/covid_19_severity_prediction/archive/refs/heads/Justin.zip" \
    -O "/tmp/chest-x-rays.zip"

zip_ref = zipfile.ZipFile('/tmp/chest-x-rays.zip', 'r') # Opens the zip file in read mode
zip_ref.extractall('/tmp') # Extracts the files into the /tmp folder
zip_ref.close()

zip_ref = zipfile.ZipFile('/tmp/covid_19_severity_prediction-Justin/Chest X-ray Image Data sets.zip', 'r') # Opens the zip file in read mode
zip_ref.extractall('/tmp') # Extracts the files into the /tmp folder
zip_ref.close()


--2022-03-25 19:51:59--  https://github.com/lukevrobbins/covid_19_severity_prediction/archive/refs/heads/Justin.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lukevrobbins/covid_19_severity_prediction/zip/refs/heads/Justin [following]
--2022-03-25 19:51:59--  https://codeload.github.com/lukevrobbins/covid_19_severity_prediction/zip/refs/heads/Justin
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/chest-x-rays.zip’

/tmp/chest-x-rays.z     [  <=>               ]   6.18M  20.0MB/s    in 0.3s    

2022-03-25 19:51:59 (20.0 MB/s) - ‘/tmp/chest-x-rays.zip’ saved [6484533]



In [6]:
# First define the settings of extractor
settings = {}
settings['label'] = 255
#settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = None
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
#settings['interpolator'] = 'sitkBSpline'
#settings['verbose'] = True
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
print('Extraction parameters:\n\t', extractor.settings)

# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.enabledImagetypes)

# Specify feature classes, all are enabled by default
extractor.disableAllFeatures()
extractor.enableAllFeatures()
#extractor.enableFeatureClassByName('firstorder')

# Specify some additional features in the GLCM feature class
#extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True}

Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': [], 'shape2D': []}


In [7]:
dataDir = os.path.join('/tmp/Chest X-ray Image Data sets')
print("dataDir, relative path:", dataDir)
print("dataDir, absolute path:", os.path.abspath(dataDir))

data = []

count = 1
for file in os.listdir(dataDir):
  if(count > len(os.listdir(dataDir))/2):
    break
  # Define the testcase name
  CXR_name = str(count)
  # Store the file paths of our testing image and label map into two variables
  imagePath = os.path.join(dataDir, CXR_name + ".jpg")
  maskPath = os.path.join(dataDir, CXR_name + "-mask.png")
  # Transforming into "3D" image for pyradiomics
  image = sitk.ReadImage(imagePath, sitk.sitkInt16)
  image3d = sitk.JoinSeries(image)
  mask = sitk.ReadImage(maskPath, sitk.sitkInt16)
  mask3d = sitk.JoinSeries(mask)

  # Extract features
  result = extractor.execute(image3d, mask3d)
  # print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
  # print('Features extracted for image ' + str(count))
  # print('Result type:', type(result))  # result is returned in a Python ordered dictionary
  # print('')
  # print('Calculated features')
  feature_value = []
  header = []
  for key, value in six.iteritems(result):
      header.append(key)
      feature_value.append(value)
  data.append(feature_value[22:])
  count = count + 1

with open("frequencies.csv", "w") as outfile:
  csvwriter = csv.writer(outfile)
  csvwriter.writerow(header[22:])
  csvwriter.writerows(data)


dataDir, relative path: /tmp/Chest X-ray Image Data sets
dataDir, absolute path: /tmp/Chest X-ray Image Data sets


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Averag